# Overview (SCALA)
## How to Use SparkSession - A Unified Entry Point in Apache Spark 2.0


In Spark 2.0, SparkSession, is a new entry point that subsumes SparkContext, SQLContext, StreamingContext, and HiveContext. For backward compatibiilty, they are preserved. SparkSession has many features, and in this notebook , by way of simple code examples, some of the more important ones, using data to illustrate its access to underlying Spark functionality. Even though, this notebook is written in Scala, similar functionality and APIs exist in Python and Java.
In DSX notebooks and Spark REPL, the SparkSession is created for you, stored in a variable called spark.

The companion blog post http://cdn2.hubspot.net/hubfs/438089/notebooks/spark2.0/SparkSession.html and https://databricks.com/blog/2016/08/15/how-to-use-sparksession-in-apache-spark-2-0.html


# PART 1: Exploring SparkSession
For backward compatibility, you can access SparkContext, SQLContext, and SparkConf

// http://www.agildata.com/apache-spark-rdd-vs-dataframe-vs-dataset/


In [79]:
spark

org.apache.spark.sql.SparkSession@f44036ea

## SparkContext as part of SparkSession
Preserved as part of SparkSession for backward compatibility.

In [2]:
spark.sparkContext

org.apache.spark.SparkContext@de6f89d8

## sqlContext as part of SparkSession
Preserved as part of SparkSession for backward compatibility

In [3]:
spark.sqlContext

org.apache.spark.sql.SQLContext@fb6615b7

# Configuring Spark's runtime configuration parameters

## SparkConf as part of SparkSession
Through spark.conf, You manipulate Spark's runtime configruation parameters. Note that all configuration options set are automatically propagated over to Spark and Hadoop during I/O.

In [4]:
spark.conf.set("spark.notebook.name", "SparkSessionSimpleZipExample")

In [5]:
spark.conf.get("spark.notebook.name")

SparkSessionSimpleZipExample

In [6]:
spark.conf.get("spark.sql.warehouse.dir")

file:/gpfs/global_fs01/sym_shared/YPProdSpark/user/sc07-a3c399a7caae2d-99fc3133bdbb/notebook/work/spark-warehouse/

## Spark config variables set can be accessed via SQL with variable subsitution

In [7]:
spark.sql("select '${spark.notebook.name}', '${spark.sql.warehouse.dir}'")

[SparkSessionSimpleZipExample: string, ${spark.sql.warehouse.dir}: string]

## Creating DataFrames and Datasets
There are a number of ways to create DataFrames and Datasets using the SparkSession APIs. Once either a DataFrame or Dataset is created, you can manipulate your data. For example, for quick exploration of Datasets, you can use the spark.range

In [8]:
import org.apache.spark.sql.functions._

In [9]:
val numDS = spark.range(5, 100, 5)
numDS.show(5)

+---+
| id|
+---+
|  5|
| 10|
| 15|
| 20|
| 25|
+---+
only showing top 5 rows



In [10]:
numDS.describe().show()

+-------+------------------+
|summary|                id|
+-------+------------------+
|  count|                19|
|   mean|              50.0|
| stddev|28.136571693556885|
|    min|                 5|
|    max|                95|
+-------+------------------+



## Creating a DataFrame from a collection with SparkSession

In [11]:
val langPercentDF = spark.createDataFrame(List(("Scala", 35), ("Python", 30), ("R", 15), ("Java", 20)))

In [12]:
val lpDF = langPercentDF.withColumnRenamed("_1", "language").withColumnRenamed("_2", "percent")

In [13]:
lpDF.orderBy(desc("percent")).show()

+--------+-------+
|language|percent|
+--------+-------+
|   Scala|     35|
|  Python|     30|
|    Java|     20|
|       R|     15|
+--------+-------+



# PART 2: Exploring Zip codes data using SparkSession and Dataset APIs.

### Next, we going to exlore some zip code data fetched from MongoDB

In [14]:
import sys.process._
"wget http://media.mongodb.org/zips.json" !

--2017-01-27 14:51:59--  http://media.mongodb.org/zips.json
Resolving media.mongodb.org (media.mongodb.org)... 52.85.202.138, 52.85.202.249, 52.85.202.70, ...
Connecting to media.mongodb.org (media.mongodb.org)|52.85.202.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3182409 (3.0M) [application/json]
Saving to: ‘zips.json.3’

     0K .......... .......... .......... .......... ..........  1%  150M 0s
    50K .......... .......... .......... .......... ..........  3% 36.0M 0s
   100K .......... .......... .......... .......... ..........  4% 37.1M 0s
   150K .......... .......... .......... .......... ..........  6% 13.9M 0s
   200K .......... .......... .......... .......... ..........  8% 99.4M 0s
   250K .......... .......... .......... .......... ..........  9% 18.5M 0s
   300K .......... .......... .......... .......... .......... 11% 19.2M 0s
   350K .......... .......... .......... .......... .......... 12% 42.8M 0s
   400K .......... .......... ....

##### The above command runs on your cluster's single node, fetches the zip code file from the specified URL, unzips in the directory below

In [15]:
"pwd" !

/gpfs/global_fs01/sym_shared/YPProdSpark/user/sc07-a3c399a7caae2d-99fc3133bdbb/notebook/work


In [16]:
"ls" !

cloudant_credentials.json
example.txt
ml-1m
ml-1m.zip
mtcars.csv
myPandasData.csv
ratings.dat
recommender_model
spark-warehouse
submission.csv
zips.json
zips.json.1
zips.json.2
zips.json.3


# Reading the JSON file with SparkSession
### Read the JSON file, infer the schema and convert it into a Dataset dictated by the case class Zips

In [2]:
import org.apache.spark.sql.SparkSession
val spark = (SparkSession.
    builder().
    getOrCreate())
// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

In [3]:
import org.apache.spark.sql.catalyst.encoders.ExpressionEncoder
import org.apache.spark.sql.Encoder
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
// A case class for zips data
case class Zips(zip:String, city:String, loc:Array[Double], pop:Long, state:String)

In [4]:
val zipDF = spark.read.json("zips.json").withColumnRenamed("_id","zip")
//rename the _id to zip for readability
//convert to a dataset using the case class
//val zipDS = zipDF.withColumnRenamed("_id","zip").as[Zips]
val zipDS = zipDF.as[Zips]
// since we will be quering this dataset often let's cache it
zipDS.cache()
//display(zipDS)
zipDS.show()

+-----+---------------+--------------------+-----+-----+
|  zip|           city|                 loc|  pop|state|
+-----+---------------+--------------------+-----+-----+
|01001|         AGAWAM|[-72.622739, 42.0...|15338|   MA|
|01002|        CUSHMAN|[-72.51565, 42.37...|36963|   MA|
|01005|          BARRE|[-72.108354, 42.4...| 4546|   MA|
|01007|    BELCHERTOWN|[-72.410953, 42.2...|10579|   MA|
|01008|      BLANDFORD|[-72.936114, 42.1...| 1240|   MA|
|01010|      BRIMFIELD|[-72.188455, 42.1...| 3706|   MA|
|01011|        CHESTER|[-72.988761, 42.2...| 1688|   MA|
|01012|   CHESTERFIELD|[-72.833309, 42.3...|  177|   MA|
|01013|       CHICOPEE|[-72.607962, 42.1...|23396|   MA|
|01020|       CHICOPEE|[-72.576142, 42.1...|31495|   MA|
|01022|   WESTOVER AFB|[-72.558657, 42.1...| 1764|   MA|
|01026|     CUMMINGTON|[-72.905767, 42.4...| 1484|   MA|
|01027|      MOUNT TOM|[-72.679921, 42.2...|16864|   MA|
|01028|EAST LONGMEADOW|[-72.505565, 42.0...|13367|   MA|
|01030|  FEEDING HILLS|[-72.675

### Q1: Can you display states, zips, cities with population greater than 40000, in descending order

In [5]:
//display(zipDS.select("state", "city", "zip", "pop").filter("pop > 40000").orderBy(desc("pop")))
zipDS.select("state", "city", "zip", "pop").filter("pop > 40000").orderBy(desc("pop")).show()

+-----+---------------+-----+------+
|state|           city|  zip|   pop|
+-----+---------------+-----+------+
|   IL|        CHICAGO|60623|112047|
|   NY|       BROOKLYN|11226|111396|
|   NY|       NEW YORK|10021|106564|
|   NY|       NEW YORK|10025|100027|
|   CA|   BELL GARDENS|90201| 99568|
|   IL|        CHICAGO|60617| 98612|
|   CA|    LOS ANGELES|90011| 96074|
|   IL|        CHICAGO|60647| 95971|
|   IL|        CHICAGO|60628| 94317|
|   CA|        NORWALK|90650| 94188|
|   IL|        CHICAGO|60620| 92005|
|   IL|        CHICAGO|60629| 91814|
|   IL|        CHICAGO|60609| 89762|
|   IL|        CHICAGO|60618| 88377|
|   NY|JACKSON HEIGHTS|11373| 88241|
|   CA|         ARLETA|91331| 88114|
|   NY|       BROOKLYN|11212| 87079|
|   CA|     SOUTH GATE|90280| 87026|
|   NY|      RIDGEWOOD|11385| 85732|
|   NY|          BRONX|10467| 85710|
+-----+---------------+-----+------+
only showing top 20 rows



### Q2: Which cities and zips in the state of california are most populous?

In [6]:
//display(zipDS.select("city", "zip", "pop").filter('state === "CA").orderBy(desc("pop")))
zipDS.select("city", "zip", "pop").filter('state === "CA").orderBy(desc("pop")).show()

+----------------+-----+-----+
|            city|  zip|  pop|
+----------------+-----+-----+
|    BELL GARDENS|90201|99568|
|     LOS ANGELES|90011|96074|
|         NORWALK|90650|94188|
|          ARLETA|91331|88114|
|      SOUTH GATE|90280|87026|
|     LOS ANGELES|90044|83958|
|         FONTANA|92335|81255|
|      HOLLY PARK|90250|78511|
|     WESTMINSTER|92683|77965|
|       SANTA ANA|92704|77151|
|        INDUSTRY|91744|77114|
|COAST GUARD ISLA|94501|76110|
|          RIALTO|92376|75341|
|     LOS ANGELES|90026|74751|
|      LONG BEACH|90805|74011|
| HUNTINGTON PARK|90255|72139|
|   MORENO VALLEY|92553|71314|
|LAKE LOS ANGELES|93550|71024|
|   SAN FRANCISCO|94110|70770|
|       IRWINDALE|91706|69464|
+----------------+-----+-----+
only showing top 20 rows



### Q3: Can you sum up the population of all the states and order them in descending order?

In [8]:
//display(zipDS.select("state", "pop").groupBy("state").sum("pop").orderBy(desc("sum(pop)")))
zipDS.select("state", "pop").groupBy("state").sum("pop").orderBy(desc("sum(pop)")).show()

+-----+--------+
|state|sum(pop)|
+-----+--------+
|   CA|29754890|
|   NY|17990402|
|   TX|16984601|
|   FL|12686644|
|   PA|11881643|
|   IL|11427576|
|   OH|10846517|
|   MI| 9295297|
|   NJ| 7730188|
|   NC| 6628637|
|   GA| 6478216|
|   VA| 6181479|
|   MA| 6016425|
|   IN| 5544136|
|   MO| 5110648|
|   WI| 4891769|
|   TN| 4876457|
|   WA| 4866692|
|   MD| 4781379|
|   MN| 4372982|
+-----+--------+
only showing top 20 rows



# PART 3: Creating Hive Table, registering a UDF, and querying it using SparkSession and Spark SQL APIs

### drop the table if one exists

In [19]:
spark.sql("DROP TABLE IF EXISTS hive_zips_table")

[]

In [20]:
import sys.process._

### Just ensure we don't have any lingering files in the directory because of eventual consistency.

In [21]:
"ls /gpfs/global_fs01/sym_shared/YPProdSpark/user/sc07-a3c399a7caae2d-99fc3133bdbb/notebook/work/spark-warehouse/hive_zips_table" !

part-r-00000-9d0f09b9-4830-4a08-a874-2749cf118332.snappy.parquet
_SUCCESS


In [22]:
 "rm -rf /gpfs/global_fs01/sym_shared/YPProdSpark/user/sc07-a3c399a7caae2d-99fc3133bdbb/notebook/work/spark-warehouse/hive_zips_table" !

In [23]:
zipDS.write.saveAsTable("hive_zips_table")

# Working and Accessing Catalog metadata

In [24]:
//display(spark.catalog.listDatabases)
spark.catalog.listDatabases.show()

+-------+----------------+--------------------+
|   name|     description|         locationUri|
+-------+----------------+--------------------+
|default|default database|file:/gpfs/global...|
+-------+----------------+--------------------+



In [25]:
//display(spark.catalog.listTables)
spark.catalog.listTables.show()

+---------------+--------+-----------+---------+-----------+
|           name|database|description|tableType|isTemporary|
+---------------+--------+-----------+---------+-----------+
|hive_zips_table| default|       null|  MANAGED|      false|
+---------------+--------+-----------+---------+-----------+



## Cache table using SparkSession API

In [26]:
spark.catalog.cacheTable("hive_zips_table")

## Q1: Can you query the Hive table with the Spark SQL query indentical to the one above Q1?

In [27]:
//display(spark.sql("SELECT state, city, zip, pop FROM hive_zips_table WHERE pop > 40000 ORDER BY pop DESC"))
spark.sql("SELECT state, city, zip, pop FROM hive_zips_table WHERE pop > 40000 ORDER BY pop DESC").show()

+-----+---------------+-----+------+
|state|           city|  zip|   pop|
+-----+---------------+-----+------+
|   IL|        CHICAGO|60623|112047|
|   NY|       BROOKLYN|11226|111396|
|   NY|       NEW YORK|10021|106564|
|   NY|       NEW YORK|10025|100027|
|   CA|   BELL GARDENS|90201| 99568|
|   IL|        CHICAGO|60617| 98612|
|   CA|    LOS ANGELES|90011| 96074|
|   IL|        CHICAGO|60647| 95971|
|   IL|        CHICAGO|60628| 94317|
|   CA|        NORWALK|90650| 94188|
|   IL|        CHICAGO|60620| 92005|
|   IL|        CHICAGO|60629| 91814|
|   IL|        CHICAGO|60609| 89762|
|   IL|        CHICAGO|60618| 88377|
|   NY|JACKSON HEIGHTS|11373| 88241|
|   CA|         ARLETA|91331| 88114|
|   NY|       BROOKLYN|11212| 87079|
|   CA|     SOUTH GATE|90280| 87026|
|   NY|      RIDGEWOOD|11385| 85732|
|   NY|          BRONX|10467| 85710|
+-----+---------------+-----+------+
only showing top 20 rows



## Q2: Find the populus cities in Calfornia with total number of zips using the hive table?

In [28]:
// display(spark.sql("SELECT COUNT(zip), SUM(pop), city FROM hive_zips_table WHERE state = 'CA' GROUP BY city ORDER BY SUM(pop) DESC"))
spark.sql("SELECT COUNT(zip), SUM(pop), city FROM hive_zips_table WHERE state = 'CA' GROUP BY city ORDER BY SUM(pop) DESC").show()

+----------+--------+----------------+
|count(zip)|sum(pop)|            city|
+----------+--------+----------------+
|        56| 2102295|     LOS ANGELES|
|        34| 1049298|       SAN DIEGO|
|        29|  816653|        SAN JOSE|
|        26|  723993|   SAN FRANCISCO|
|        28|  628279|      SACRAMENTO|
|        12|  347905|          FRESNO|
|        12|  314487|         OAKLAND|
|         8|  299651|      LONG BEACH|
|         7|  272327|         ANAHEIM|
|         8|  271347|     BAKERSFIELD|
|        11|  267258|        STOCKTON|
|         7|  253478|       RIVERSIDE|
|         4|  234472|       SANTA ANA|
|         5|  216459|         MODESTO|
|         4|  183542|HUNTINGTON BEACH|
|         7|  177552|  SAN BERNARDINO|
|         4|  173374|         FREMONT|
|         8|  163666|        GLENDALE|
|         6|  158398|      SANTA ROSA|
|         6|  158183|        TORRANCE|
+----------+--------+----------------+
only showing top 20 rows



# Registring a UDF with SparkSession

## Q4: Can you register a simple UDF with SparkSession that converts zip into long (currently it's a string)?

In [29]:
spark.sql("describe hive_zips_table").show()

+--------+-------------+-------+
|col_name|    data_type|comment|
+--------+-------------+-------+
|     zip|       string|   null|
|    city|       string|   null|
|     loc|array<double>|   null|
|     pop|       bigint|   null|
|   state|       string|   null|
+--------+-------------+-------+



In [30]:
spark.udf.register("zipToLong", (z:String) => z.toLong)

UserDefinedFunction(<function1>,LongType,Some(List(StringType)))

In [31]:
spark.sql("SELECT city, zipToLong(zip) as zip_to_long FROM hive_zips_table ORDER BY zip_to_long DESC").show()

+-----------+-----------+
|       city|zip_to_long|
+-----------+-----------+
|  KETCHIKAN|      99950|
|   WRANGELL|      99929|
|POINT BAKER|      99927|
| METLAKATLA|      99926|
|    KLAWOCK|      99925|
|      HYDER|      99923|
|   HYDABURG|      99922|
|      CRAIG|      99921|
| THORNE BAY|      99919|
|  KETCHIKAN|      99901|
|    SKAGWAY|      99840|
|      SITKA|      99835|
| PETERSBURG|      99833|
|     HOONAH|      99829|
|     HAINES|      99827|
|   GUSTAVUS|      99826|
|    DOUGLAS|      99824|
|     ANGOON|      99820|
|     JUNEAU|      99801|
|    NUIQSUT|      99789|
+-----------+-----------+
only showing top 20 rows



### Register another UDF that calculates the strlen of cities

In [32]:
spark.udf.register("cityLength", (c:String) => c.length())

UserDefinedFunction(<function1>,IntegerType,Some(List(StringType)))

### Using catalog data, get the list of your registered UDFs

In [33]:
val udfs = spark.catalog.listFunctions()

In [34]:
udfs.filter('name === "cityLength".toLowerCase || 'name === "zipToLong".toLowerCase).select("name", "database").show()

+----------+--------+
|      name|database|
+----------+--------+
|citylength|    null|
| ziptolong|    null|
+----------+--------+



In [103]:
spark.sql("SELECT city, cityLength(city) as city_length FROM hive_zips_table ORDER BY city_length DESC").show()

+----------------+-----------+
|            city|city_length|
+----------------+-----------+
|CHEBEAGUE ISLAND|         16|
|MONTGOMERY CENTE|         16|
|CUMBERLAND CENTE|         16|
|WEST BRIDGEWATER|         16|
|OLD ORCHARD BEAC|         16|
|WEST SPRINGFIELD|         16|
|CUMBERLAND FORES|         16|
|GREAT BARRINGTON|         16|
|NORTH WHITEFIELD|         16|
|NORTH CHELMSFORD|         16|
|EAST MILLINOCKET|         16|
|NEWTON UPPER FAL|         16|
|GREENVILLE JUNCT|         16|
|GILMANTON IRON W|         16|
|LITTLE DEER ISLE|         16|
|WOOD RIVER JUNCT|         16|
|SOUTH GOULDSBORO|         16|
|CENTER BARNSTEAD|         16|
|SOUTHWEST HARBOR|         16|
|WEST CHESTERFIEL|         16|
+----------------+-----------+
only showing top 20 rows



### Q5: Can you compose the same query as Q2 using Datasets APIs?

In [106]:
(zipDS.filter('state === "CA")
  .select("zip", "pop", "city")
  .groupBy("city")
  .sum()
  .orderBy(desc("sum(pop)")).show())

+----------------+--------+
|            city|sum(pop)|
+----------------+--------+
|     LOS ANGELES| 2102295|
|       SAN DIEGO| 1049298|
|        SAN JOSE|  816653|
|   SAN FRANCISCO|  723993|
|      SACRAMENTO|  628279|
|          FRESNO|  347905|
|         OAKLAND|  314487|
|      LONG BEACH|  299651|
|         ANAHEIM|  272327|
|     BAKERSFIELD|  271347|
|        STOCKTON|  267258|
|       RIVERSIDE|  253478|
|       SANTA ANA|  234472|
|         MODESTO|  216459|
|HUNTINGTON BEACH|  183542|
|  SAN BERNARDINO|  177552|
|         FREMONT|  173374|
|        GLENDALE|  163666|
|      SANTA ROSA|  158398|
|        TORRANCE|  158183|
+----------------+--------+
only showing top 20 rows



In [69]:
(zipDS.filter('state === "CA")
  .select("zip", "pop", "city")
  .groupBy("city")
  .agg(sum("pop").alias("population"))
  .orderBy(desc("population"))).show()

+----------------+----------+
|            city|population|
+----------------+----------+
|     LOS ANGELES|   2102295|
|       SAN DIEGO|   1049298|
|        SAN JOSE|    816653|
|   SAN FRANCISCO|    723993|
|      SACRAMENTO|    628279|
|          FRESNO|    347905|
|         OAKLAND|    314487|
|      LONG BEACH|    299651|
|         ANAHEIM|    272327|
|     BAKERSFIELD|    271347|
|        STOCKTON|    267258|
|       RIVERSIDE|    253478|
|       SANTA ANA|    234472|
|         MODESTO|    216459|
|HUNTINGTON BEACH|    183542|
|  SAN BERNARDINO|    177552|
|         FREMONT|    173374|
|        GLENDALE|    163666|
|      SANTA ROSA|    158398|
|        TORRANCE|    158183|
+----------------+----------+
only showing top 20 rows



In [71]:
(zipDS.filter('state === "CA")
  .select("zip", "pop", "city")
  .groupBy("city")
  .agg(sum("pop").alias("population"), count("*").alias("cnt"))
  .orderBy(desc("population"))).show()

+----------------+----------+---+
|            city|population|cnt|
+----------------+----------+---+
|     LOS ANGELES|   2102295| 56|
|       SAN DIEGO|   1049298| 34|
|        SAN JOSE|    816653| 29|
|   SAN FRANCISCO|    723993| 26|
|      SACRAMENTO|    628279| 28|
|          FRESNO|    347905| 12|
|         OAKLAND|    314487| 12|
|      LONG BEACH|    299651|  8|
|         ANAHEIM|    272327|  7|
|     BAKERSFIELD|    271347|  8|
|        STOCKTON|    267258| 11|
|       RIVERSIDE|    253478|  7|
|       SANTA ANA|    234472|  4|
|         MODESTO|    216459|  5|
|HUNTINGTON BEACH|    183542|  4|
|  SAN BERNARDINO|    177552|  7|
|         FREMONT|    173374|  4|
|        GLENDALE|    163666|  8|
|      SANTA ROSA|    158398|  6|
|        TORRANCE|    158183|  6|
+----------------+----------+---+
only showing top 20 rows



In [74]:
(zipDS.filter('state === "CA")
  .select("zip", "pop", "city")
  .groupBy("city")
  .agg(sum("pop").alias("population"), count("zip").alias("zip"))
  .orderBy(desc("population"))).show()

+----------------+----------+---+
|            city|population|zip|
+----------------+----------+---+
|     LOS ANGELES|   2102295| 56|
|       SAN DIEGO|   1049298| 34|
|        SAN JOSE|    816653| 29|
|   SAN FRANCISCO|    723993| 26|
|      SACRAMENTO|    628279| 28|
|          FRESNO|    347905| 12|
|         OAKLAND|    314487| 12|
|      LONG BEACH|    299651|  8|
|         ANAHEIM|    272327|  7|
|     BAKERSFIELD|    271347|  8|
|        STOCKTON|    267258| 11|
|       RIVERSIDE|    253478|  7|
|       SANTA ANA|    234472|  4|
|         MODESTO|    216459|  5|
|HUNTINGTON BEACH|    183542|  4|
|  SAN BERNARDINO|    177552|  7|
|         FREMONT|    173374|  4|
|        GLENDALE|    163666|  8|
|      SANTA ROSA|    158398|  6|
|        TORRANCE|    158183|  6|
+----------------+----------+---+
only showing top 20 rows

